# RPN

Afegim a la U-Net una branca nova, la branca de *region proposal network (RPN)*.  Introduida per primer cop per la *faster rcnn* duu a terme dues tasques alhora, per una part refina tot un conjunt de <a hfre="https://www.termcat.cat/ca/cercaterm/bounding%20box?type=basic">envolupants </a> i per l'altra indica quina és la probabilitat que cada un d'ells contengui un objecte.

<img style="width:75%" src="https://tryolabs.com/blog/images/blog/post-images/2018-01-18-faster-rcnn/rpn-conv-layers.63c5bf86.png" />

### Importam llibreries

In [1]:
import os
import json

from numpy.random import seed
import numpy as np
import skimage
import skimage.color
import skimage.io
import skimage.transform


# Llibraries pròpies
from u_cells.u_cells.rpn import data as rpn_data
from u_cells.u_cells.unet import model as u_model
from u_cells.u_cells.common import config as rpn_config

seed(1)

### Configuració

Primerament cream un classe configuració per l'execusió i entrenament de la xarxa. En aquesta classe deixam els valors per defecte exceptuant els casos del nombre de classes, la mida de les ancores, les pases per època i el llindar mínim de confiança.

In [2]:
MULTI_CLASS = False

class CellConfig(rpn_config.Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "cells"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.

    # Number of classes (including background)
    if MULTI_CLASS:
        NUM_CLASSES = 1 + 3  # Background + 3 classes
    else:
        NUM_CLASSES = 1 + 1  # Background + 3 classes
    
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0
    
    IMAGE_SHAPE = [512, 512, 3]
    
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_DIM = 400


config = CellConfig()
config.IMAGE_SHAPE = np.array([512,512,3])

print(config)

Configurations:
BACKBONE_STRIDES               [4, 8, 16, 32, 64] 
BATCH_SIZE                     2 
DETECTION_MIN_CONFIDENCE       0 
GRADIENT_CLIP_NORM             5.0 
IMAGE_CHANNEL_COUNT            3 
IMAGE_MAX_DIM                  512 
IMAGE_META_SIZE                14 
IMAGE_MIN_DIM                  400 
IMAGE_MIN_SCALE                0 
IMAGE_RESIZE_MODE              square 
IMAGE_SHAPE                    [512 512   3] 
LEARNING_RATE                  3e-05 
MASK_POOL_SIZE                 14 
MAX_GT_INSTANCES               100 
MEAN_PIXEL                     [123.7 116.8 103.9] 
NAME                           cells 
NUM_CLASSES                    2 
POOL_SIZE                      7 
ROI_POSITIVE_RATIO             0.33 
RPN_ANCHOR_RATIOS              [0.5, 1, 2] 
RPN_ANCHOR_SCALES              (8, 16, 32, 64, 128) 
RPN_ANCHOR_STRIDE              1 
RPN_BBOX_STD_DEV               [0.1 0.1 0.2 0.2] 
RPN_TRAIN_ANCHORS_PER_IMAGE    256 
STEPS_PER_EPOCH                100 
TRAIN_ROIS_P

## Entrenament

Per realitzar l'entrenament primerament cream dos generadors d'imatges. Els generadors en el cas de la *RPN* es creen en dos temps. Primerament cream objectes **Dataset**.

### Dataset

Definim un objecte Dataset. Anàlogament a la configuració, ja definida, és basa en herència de classes abstractes definides a les llibreries. Un detall important és que en el cas de la RPN les dades es formen a partir dels envolupants, enlloc de l'inrevés.

In [3]:
class CellDataset(rpn_data.Dataset):

    def load_cell(self, dataset_dir, subset):
        """Load a subset of the Erithocites2 dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        if MULTI_CLASS:
            self.add_class("cell", 1, "ELONGATED")
            self.add_class("cell", 2, "CIRCULAR")
            self.add_class("cell", 3, "OTHER")
        else:
            self.add_class("cell", 1, "cell")

        
        # Train or validation dataset?
        print(subset)
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Anottation following the format oof VIA
        annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                a['regions'] = a['regions'].values()
                
            aux = [(r['shape_attributes'], r['type'] + 1) for r in a['regions']]
            
            polygons, cells = list(zip(*aux))
            
            
            if not MULTI_CLASS:
                cells = np.ones([len(polygons)], dtype=np.int32)
            else:
                cells = np.asarray(cells)

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "cell",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons, cells = cells)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
        
        Args:
            image_id:
        
        Returns:
            masks:  A bool array of shape [height, width, instance count] with
                    one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        gt_class = []
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), info["cells"]
        

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        return info["path"]

Una vegada definit el *dataset* cream dues instàncies, una per l'entrenament i l'altra per validació.

In [4]:
# Training dataset.
dataset_train = CellDataset()
dataset_train.load_cell("./in/bboxes_class/", "train")
dataset_train.prepare()

# Validation dataset
dataset_val = CellDataset()
dataset_val.load_cell("./in/bboxes_class/", "val")
dataset_val.prepare()

train
val


In [5]:
train_generator = rpn_data.DataGenerator(100, dataset_train, config, shuffle=True)
val_generator = rpn_data.DataGenerator(100, dataset_val, config, shuffle=True)

### Construim el model


In [6]:
model = u_model.UNet(input_size=(512,512,3), out_channel=100, batch_normalization=True, 
                     rpn=True, config_net = config)

model.build_unet(n_filters=64, dilation_rate=1, last_activation="softmax")
model.compile(run_eagerly=True)
print(model.model.summary())

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Tensor("Placeholder_1:0", shape=(None, None, 1), dtype=int32)
Model: "r-unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv_block (ConvBlock)          (None, 512, 512, 64) 39232       input_image[0][0]                
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 64) 0

### Entrenam el model

In [7]:
model.train(train_generator=train_generator, val_generator=val_generator, epochs=10, steps_per_epoch=100, 
            validation_steps=20, check_point_path=None)

Epoch 1/10
tf.Tensor(
[[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]], shape=(2, 65472, 1), dtype=int32)


InvalidArgumentError: slice index 2 of dimension 0 out of bounds. [Op:StridedSlice] name: r-unet/rpn_bbox_loss/strided_slice/

In [8]:
for m, _ in train_generator:
    break

In [24]:
m[1].shape

(2, 65472, 1)

In [23]:
m[2].shape

(2, 256, 4)